In [3]:
import requests as req
import bs4 as bs

In [1]:
class AdelphiScraper():
  def __init__(self):
    self.catalog      = 'https://www.adelphi.it/catalogo/'
    self.subject      = 'https://www.adelphi.it/catalogo/materia/'
    self.subjects_map = self.init_subjects()

  def init_subjects(self):
    resp  = req.get(self.catalog)
    soup  = bs.BeautifulSoup(resp.text, "html.parser")
    items = soup.find_all('ul', class_='subject')
    ret = dict()
    for i in items:
      for j in i:
        ret.update({int(j.a['href'].split('/')[-1]) : j.text})
    return ret

  def get_subject_list(self):
    return self.subjects_map
  
  def get_subject_books(self, subject_code):
    page_found = True
    suffix = ''
    page_counter = 1
    books = dict()
    isbns = dict()
    while page_found:
      try:
        resp = req.get(self.subject + str(subject_code) + suffix, allow_redirects = False)
        soup = bs.BeautifulSoup(resp.text, "html.parser")
        if not len(soup):
          raise ValueError()
        book_infos, isbn_maps = self.extract_books(soup)
        books.update(book_infos)
        isbns.update(isbn_maps)
        page_counter += 1
        suffix = '/p' + str(page_counter)
      except:
        page_found = False
    return books, isbns

  def extract_books(self, soup):
    items = soup.find_all('div', class_='list_impressum')
    items2 = soup.find_all('div', class_='data' )
    items3 = soup.find_all('div', class_ = "abstract hidden-xs")
    isbn_map  = dict()
    book_info = dict()
    counter = 0
    for i in items:
      isbn_map.update({i.a['href'].split('/')[-1] : i.a.text}) #isbn : titolo
      book_info.update({i.a.text : {'Autore': i.div.text, \
                                    'Anno': items2[counter].text.split(' ')[0].split(',')[0], \
                                    'Prezzo Originale' : i.span.text.split(' ')[1].replace(',','.'), \
                                    'Sconto': i.span.text.split(' ')[2], \
                                    'Pagine' :items2[counter].text.split(' ')[2].split(',')[0].split('-')[-1], \
                                    'Abstract': items3[counter].text}})
      counter += 1
    return book_info, isbn_map

In [4]:
# creo data = dict con associato all'ISBN ogni dato del libro. (tempo di compilazione lungo ~10min)
Ad_Scraper = AdelphiScraper() 
subj = Ad_Scraper.get_subject_list()
data = {}
for i in range(len(subj)):
    books = Ad_Scraper.get_subject_books(i+1)
    for ISBN in books[1].keys():
        titolo = books[1][ISBN]
        anno = books[0][titolo]["Anno"]
        autore = books[0][titolo]["Autore"]
        riassunto = books[0][titolo]["Abstract"]
        pagine = books[0][titolo]["Pagine"]
        prezzo = books[0][titolo]["Prezzo Originale"]
        sconto = eval(((books[0][titolo]["Sconto"]).replace('-','')).replace('%',''))
        prezzo_scontato = eval(prezzo) * (100 - sconto) / 100
        categoria = subj[i+1]
        data.update({ISBN : {"Titolo" : titolo, "Anno" : anno, "Autore" : autore, "Riassunto" : riassunto,\
                        "Pagine" : pagine, "Prezzo" : prezzo, "Prezzo_scontato" : prezzo_scontato, \
                        "Categoria" : categoria}})

print("fine")

fine


In [ ]:
# Attingo dalla lista data che contiene le informazioni di ogni libro dell'Adelphi per velocizzare 
# il codice
class Book():
    def __init__(self, ISBN):
        self.ISBN = ISBN
        self.title = data[str(ISBN)]["Titolo"]
        self.print_year = data[str(ISBN)]["Anno"] 
        self.author = data[str(ISBN)]["Autore"]
        self.abstract = data[str(ISBN)]["Riassunto"]
        self.pages = data[str(ISBN)]["Pagine"]
        self.category = data[str(ISBN)]["Categoria"]
        self.price = data[str(ISBN)]["Prezzo"]
        self.sale = data[str(ISBN)]["Prezzo_scontato"]
    
    def get_book_infos(self):  #Scrive le info del libro in un dict
        info = {"ISBN" : self.ISBN, "Titolo" : self.title, "Anno" : self.print_year, \
                "Autore" : self.author, "Riassunto" : self.abstract, "Pagine" : self.pages,\
                "Categoria" : self.category, "Prezzo" : self.price, "Prezzo scontato" : self.sale}
        return info
        
    def print_book_infos(self):
        info = self.get_book_infos()
        for keys, values in info.items():
            print(str(keys) +": "+str(values))
            
    def get_whole_price(self): # Prezzo pieno
        return str(self.price)
    
    def get_net_price(self): # Prezzo scontato
        return str(self.sale)
    
    def abstract_len(self): 
        return len(self.abstract)
    
    def is_near(self, book_2):
        if (self.category == book_2.category) or ((self.ISBN % 3) == (book_2.ISBN % 3)):
            return True
        else:
            return False

In [ ]:
class MyLibrary():
    def __init__(self,ISBN):
        self.library = []
        self.libri = self.create_library()
    
    def create_library(self): # Creo la libreria inserendo i libri non come classe ma come dict con le info
        for i in range(len(ISBN)):
            if type(ISBN == str):
                libro = Book(eval(ISBN[i]))
            else:
                libro = Book(ISBN[i])
            self.library.append(libro.get_book_infos())
    
    def show_library(self): # Stampo la libreria
        print("------------- Inizio libreria -------------")
        for i in range(len(self.library)):
            info = self.library[i]
            for keys, values in info.items():
                print(str(keys) +": "+str(values))
            
            print()
        print("------------- Fine libreria -------------")
        print()
    
    def insert_book(self, ISBN): # Aggiungo libro
        libro = Book(ISBN)
        self.library.append(libro.get_book_infos())
    
    def remove_book(self, ISBN): # Rimuovo libro
        for i in range(len(self.library)): 
            if self.library[i]['ISBN'] == ISBN: 
                del self.library[i] 
                break
    
    def get_library_value(self):
        """Output_1: prezzo pieno; Output_2: prezzo scontato"""
        prezzo = 0
        prezzo_scont = 0
        for i in range(len(self.library)):
            prezzo += eval(self.library[i]["Prezzo"])
            prezzo_scont += self.library[i]["Prezzo scontato"]
        
        return prezzo, prezzo_scont

    def get_total_pages(self):
        """Output numero pagine"""
        pagine = 0
        for i in range(len(self.library)):
            pagine += eval(self.library[i]["Pagine"])
        
        return pagine
    
    def get_categories_number(self):
        """Output_1: numero di categorie; Output_2: elenco delle categorie"""
        cat = []
        cat_number = 0
        for i in range(len(self.library)):
            if self.library[i]["Categoria"] not in cat:
                cat_number += 1
                cat.append(self.library[i]["Categoria"])
            
        return cat_number,cat

In [ ]:
ISBN = ['9788845912207', '9788845912023']
        #'9788845911972', '9788845911415', '9788845911446','9788845911408', '9788845911156'
#Prove dei metodi inclusi
lib = MyLibrary(ISBN)
lib.show_library()
lib.insert_book(9788845911415)
lib.show_library()
lib.remove_book(9788845912207)
lib.show_library()
prezzo_l, prezzo_n = lib.get_library_value()
print("Prezzo lordo:",prezzo_l,"--- Prezzo netto:",prezzo_n)
pagine = lib.get_total_pages()
print("Totale pagine:", pagine)
number, categ = lib.get_categories_number()
print("Numero di categorie: ",number)
for i in range(len(categ)):
    print(i+1,"-",categ[i],end=";\t")

------------- Inizio libreria -------------
ISBN: 9788845912207
Titolo: Anatomia dell’irrequietezza
Anno: 1999
Autore: Bruce Chatwin
Riassunto: Per i lettori che conoscono e amano Bruce Chatwin Anatomia dell’irrequietezza diventerà una sorta di breviario, per quelli che hanno meno familiarità con l’opera del grande vagabondo sarà un’introduzione alle sue esperienze e alle sue idee, un viaggio con Chatwin alla scoperta di Chatwin. Forse mai come in certe parti di questo libro – soprattutto nelle notizie...
Pagine: 304
Categoria: Ritratti
Prezzo: 14.00
Prezzo scontato: 13.3

ISBN: 9788845912023
Titolo: La ballata di Peckham Rye
Anno: 1996
Autore: Muriel Spark
Riassunto: Pubblicato nel 1960, ai tempi delle più pensose e tediose divagazioni sulla «letteratura industriale», questo romanzo le doppiava fulmineamente con uno sberleffo, proiettandosi in un paesaggio che molto somiglia a quello di oggi. Scandendo i tempi di una «ballata» sinistra ed euforica, la Spark ci racconta come la direzio